<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/CNN-LSTM-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, Dense, MaxPooling1D, LSTM
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [4]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv',chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [7]:
model = Sequential()
model.add(Conv1D(activation="tanh", input_shape=(len(test.columns)-1,1), kernel_size=2, filters=100))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, activation="tanh", recurrent_activation="sigmoid"))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 69, 100)           300       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 34, 100)          0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 80,801
Trainable params: 80,801
Non-trainable params: 0
_________________________________________________________________


In [8]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x, train_y, batch_size=32)

6161/6161 [==============================] - 32s 5ms/step - loss: 2.8814e-04 - accuracy: 0.9999


In [9]:
test_y = test.pop('Label')
test_x = test

In [10]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [16]:
pred = model.predict(test_x, verbose=1)
pred_y = np.round(pred).astype(int)

79666/79666 [==============================] - 217s 3ms/step


In [17]:
confusion_matrix(test_y, pred_y)

array([[1255517,     224],
       [      1, 1293546]])

In [19]:
report = classification_report(test_y, pred_y, digits=6)
print(report)

              precision    recall  f1-score   support

           0   0.999999  0.999822  0.999910   1255741
           1   0.999827  0.999999  0.999913   1293547

    accuracy                       0.999912   2549288
   macro avg   0.999913  0.999910  0.999912   2549288
weighted avg   0.999912  0.999912  0.999912   2549288

